In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('..\\2. dane\\Walmart.csv')

In [3]:
df['Date'] = pd.to_datetime(df['Date'])
df['yyyy'] = pd.to_datetime(df['Date']).dt.year
df['mm'] = pd.to_datetime(df['Date']).dt.month
df['dd'] = pd.to_datetime(df['Date']).dt.day

all_weeks = pd.Series(
    pd.date_range(start='2010-01-01', end='2012-12-31', freq='7D'))
dfs = pd.DataFrame(data=all_weeks, columns=["Date"])

In [4]:
def res_builder(df1):
    res = pd.concat([df1, dfs])
    res['yyyy'] = pd.to_datetime(res['Date']).dt.year
    res['mm'] = pd.to_datetime(res['Date']).dt.month
    res['dd'] = pd.to_datetime(res['Date']).dt.day
    res = res.sort_values(["Date"]).reset_index()
    res['week_absolute'] = res.Date.apply(lambda x: x.weekofyear)
    res = res.drop(['Date', 'index'], axis=1)
    return res

In [5]:
def dff_builder(res):
    
    dff = pd.DataFrame()
    
    for x in range(0, (len(res) - 1)):
        # saving empty inserts
        if res['week_absolute'][x] != res['week_absolute'][x + 1]:
            dftmp = res.loc[(res['week_absolute'] == res['week_absolute'][x])
                            & (res['yyyy'] == res['yyyy'][x])]
            dff = dff.append(dftmp)
        # saving original inserts
        elif res['week_absolute'][x] == res['week_absolute'][x + 1]:
            dftmp = res.loc[(res['week_absolute'] == res['week_absolute'][x])
                            & (res['yyyy'] == res['yyyy'][x])]
            # calculating the average
            for col in dftmp.columns:
                dftmp[col][x] = np.nanmean(dftmp[col])
                # cleaning temporar dataframe
                for l in range(len(dftmp)):
                    dftmp[col][x + l] = dftmp[col][x]
            dftmp = dftmp.drop_duplicates()
            dftmp = dftmp.iloc[[0]]
            dff = dff.append(dftmp)


    # adding missing info
    dff = dff.append(res.iloc[[-1]])
    dff = dff.reset_index()
    dff = dff.drop(['index'], axis=1)
    # groupping
    dff = dff.groupby(['week_absolute', 'yyyy']).first()
    dff = dff.reset_index()
    dff = dff.sort_values(['yyyy', 'week_absolute']).reset_index()
    dff = dff.drop(['index'], axis=1)
    # adding relative week
    dff['week_relative'] = np.nan
    dff['week_relative'][0] = 0
    tmp = 0
    for x in range(1, (len(dff) - 1)):
        tmp += 1
        dff['week_relative'][x] = tmp
    dff['week_relative'][len(dff) -
                         1] = dff['week_relative'][(len(dff) - 2)] + 2
    # interpolating
    dff = dff.interpolate()
    dff['Holiday_Flag'] = dff['Holiday_Flag'].round()
    # return
    return dff

In [6]:
def DF_creator():
    DF = pd.DataFrame()
    # establishing the DF
    for i in range(1, (len(df.Store.unique())) + 1):
        dfi = df.loc[df['Store'] == i]
        DF = DF.append(dff_builder(res_builder(dfi)))

    # cleaning DF
    DF.reset_index(inplace=True)
    DF.drop(columns=['index'])
    DF = DF[[
        'Store', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price',
        'CPI', 'Unemployment', 'yyyy', 'mm', 'dd', 'week_relative',
        'week_absolute'
    ]]
    DF['Weekly_Sales'] = DF['Weekly_Sales'].round()
    DF[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'
        ]] = DF[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']].round(3)
    return DF

In [7]:
DF = DF_creator()
DF.to_csv('fail_data_analysis.csv')

**---------------------------------------------------------------------------------------------------------------**

In [8]:
DF

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,yyyy,mm,dd,week_relative,week_absolute
0,1.0,1453330.0,0.0,71.890,2.603,211.672,7.838,2010,1,9,0.0,1
1,1.0,1475920.0,0.0,71.815,2.626,211.509,7.828,2010,1,15,1.0,2
2,1.0,1498511.0,0.0,71.740,2.649,211.347,7.818,2010,1,22,2.0,3
3,1.0,1521102.0,0.0,71.665,2.671,211.184,7.808,2010,1,29,3.0,4
4,1.0,1543693.0,0.0,71.590,2.694,211.022,7.797,2010,2,5,4.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
7060,45.0,748874.0,0.0,57.178,3.956,191.787,8.627,2012,11,30,152.0,48
7061,45.0,741669.0,0.0,55.824,3.978,192.057,8.647,2012,12,7,153.0,49
7062,45.0,734464.0,0.0,54.470,4.000,192.327,8.667,2012,12,12,154.0,50
7063,45.0,734464.0,0.0,54.470,4.000,192.327,8.667,2012,12,21,155.0,51
